[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/atmos-cloud-sim-uj/PySDM-examples.git/main?urlpath=PySDM_examples/Lowe_et_al_2019/fig_2.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atmos-cloud-sim-uj/PySDM-examples/blob/main/PySDM_examples/Lowe_et_al_2019/fig_2.ipynb)

#### based on Fig. 2 from Lowe et al. 2019 (Nature Comm.)  "_Key drivers of cloud response to surface-active organics_"   
https://doi.org/10.1038/s41467-019-12982-0

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install git+https://github.com/atmos-cloud-sim-uj/PySDM-examples.git
    !ldconfig

In [2]:
from PySDM_examples.Lowe_et_al_2019 import Settings, Simulation
from PySDM_examples.utils.show_plot import show_plot
from PySDM.physics import si
import numpy as np
from matplotlib import pyplot

ModuleNotFoundError: No module named 'PySDM_examples'

In [3]:
settings = Settings(dt=.25*si.s, n_sd=20, n_substep=1)
settings.output_interval = 25 * si.s

In [4]:
simulation = Simulation(settings)

In [5]:
output = simulation.run()

In [3]:
figsize = (10, 6)
fig, axs = pyplot.subplots(1, 3, figsize=figsize, sharey=True)

Y = np.asarray(output['t']) - 196 * si.s

axs[0].plot(output['ql'], Y)
axs[0].set_ylim(0, 2400 * si.s)
axs[0].set_xlim(0, 2.5)
axs[0].set_yticks(np.linspace(0, 2400, 13, endpoint=True))
axs[0].set_ylabel('Time above cloud base (s)')
axs[0].set_xlabel('Liquid Water Content (g/kg)')

for ax in axs:
    ax.grid()
show_plot()

NameError: name 'pyplot' is not defined